 Сухоруков Кирилл Евгеньевич

# Импорт библиотек

In [41]:
import numpy as np
import pandas as pd
import sklearn
from scipy import sparse
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import normalize

# Подгрузка файла с Google Drive

In [42]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [43]:
users_filename = 'lastfm_user_scrobbles.csv'
artists_filename = 'lastfm_artist_list.csv'
work_dir='/content/drive/My Drive/Colab Notebooks/'
users_filepath = work_dir + users_filename
artists_filepath = work_dir + artists_filename

# Чтение и предобработка данных

In [44]:
users_data = pd.read_csv(users_filepath)
artists_data = pd.read_csv(artists_filepath)

In [45]:
print(users_data)

       user_id  artist_id  scrobbles
0            1       4562      13883
1            1      10191      11690
2            1        494      11351
3            1       6673      10300
4            1       8402       8983
...        ...        ...        ...
92787     1892      10098        278
92788     1892       8660        263
92789     1892       3274        258
92790     1892       4240        232
92791     1892      10842        228

[92792 rows x 3 columns]


In [46]:
print(artists_data)

       artist_id   artist_name
0              1       __Max__
1              2       _Algol_
2              3     -123 Min.
3              4          -Oz-
4              5  -T De Sangre
...          ...           ...
17488      17489          鷺巣詩郎
17489      17490           黃立行
17490      17491           黄义达
17491      17492         黒木メイサ
17492      17493            鼓童

[17493 rows x 2 columns]


In [47]:
users_artists_table = pd.merge(users_data, artists_data, on='artist_id')
print(users_artists_table)

       user_id  artist_id  scrobbles          artist_name
0            1       4562      13883          Duran Duran
1            3       4562        228          Duran Duran
2           25       4562         85          Duran Duran
3           26       4562         10          Duran Duran
4           60       4562        528          Duran Duran
...        ...        ...        ...                  ...
92787     1892      10879        337            Nyktalgia
92788     1892       1357        297      Atsakau Niekada
92789     1892       4360        281   Domantas Razauskas
92790     1892       1330        280              Atalyja
92791     1892       8660        263  Les Chants De Nihil

[92792 rows x 4 columns]


In [48]:
users_artists_table = users_artists_table.drop(columns='artist_id')
users_artists_table = users_artists_table.reindex(columns=['user_id', 
                                                           'artist_name', 
                                                           'scrobbles'])
print(users_artists_table)

       user_id          artist_name  scrobbles
0            1          Duran Duran      13883
1            3          Duran Duran        228
2           25          Duran Duran         85
3           26          Duran Duran         10
4           60          Duran Duran        528
...        ...                  ...        ...
92787     1892            Nyktalgia        337
92788     1892      Atsakau Niekada        297
92789     1892   Domantas Razauskas        281
92790     1892              Atalyja        280
92791     1892  Les Chants De Nihil        263

[92792 rows x 3 columns]


# Преобразование скробблов

In [49]:
def conditional_scrobbles_score(table: pd.DataFrame): 
  '''Функция для сглаживания значений скробблов у пользователей

  Parameters
  @table: pandas.DataFrame
  Таблица пользователей и прослушиваемых ими артистов, с количеством 
  прослушиваний артистов этими пользователями

  Returns
  @result: pandas.DataFrame

  '''
  result = table.copy()
  user_ids = table['user_id'].unique()
  for i in user_ids:
    slice_members = result['user_id'] == i
    users_scrobbles_slice = result[slice_members]
    users_avg_scrobble = users_scrobbles_slice['scrobbles'].mean()
    users_scrobbles_sum = users_scrobbles_slice['scrobbles'].sum()
    bias_condition = users_scrobbles_slice['scrobbles'] > 3*users_avg_scrobble
    biased_scrobbles = users_scrobbles_slice[bias_condition]['scrobbles']\
                                                  .sub(users_avg_scrobble)
    scaled = users_scrobbles_slice['scrobbles'].div(users_scrobbles_sum)
    scaled.loc[bias_condition] = biased_scrobbles.div(users_scrobbles_sum)
    result.loc[slice_members, 'scrobbles'] = scaled
  result['scrobbles'].apply(float)
  return result

In [50]:
users_artists_table = users_artists_table.sort_values(by='user_id')

In [51]:
conditional_users_artists_table = conditional_scrobbles_score(
    users_artists_table)

In [52]:
print(conditional_users_artists_table)

       user_id          artist_name  scrobbles
0            1          Duran Duran   0.062276
3380         1    Jean-Michel Jarre   0.008338
3396         1    Ministry Of Sound   0.008137
3398         1           Simply Red   0.008078
3411         1        Fleetwood Mac   0.007953
...        ...                  ...        ...
79687     1892                Anubi   0.020727
79658     1892     Altorių Šešėliai   0.019853
79656     1892    Satanic Warmaster   0.020274
39017     1892            Moonspell   0.013084
92791     1892  Les Chants De Nihil   0.008518

[92792 rows x 3 columns]


# Реализация Item-based рекомендательной системы

In [53]:
def find_favorite_artists(user_id: int, 
                          amount: int=5):
  """Найти любимых исполнителей у пользователя

  Parameters
  :param user_id: int - ID пользователя
  :param amount: int (default 5) - Количество искомых исполнителей
  
  Return
  :return result: list - Список любимых исполнителей пользователя
  """
  slice_condition = users_artists_table['user_id'] == user_id
  user_slice = users_artists_table[slice_condition]
  result = user_slice.sort_values('scrobbles', ascending=False).head(amount)
  return result['artist_name'].tolist()

In [54]:
def get_similar_artists_sparse(artist_name: str, 
                               amount: int=5):
  """Найти артистов, похожих на указанного

  Parameters
  :param artist_name: str - Артист, для которого ищутся похожие на него артисты
  :param amount: int (default 5) - Количество искомых артистов

  Return
  :return result: list - Список найденных артистов
  """
  condition = artists_data['artist_name']==artist_name
  col_idx = artists_data[condition].index[0]
  corr_specific = sim[col_idx]
  result = [title_dict[i+1] for i in corr_specific.toarray()\
            .argsort()[0][-amount::]]
  return result

In [55]:
def get_recommendation_sparse(user_id: int, 
                              fav_artists: int=5, 
                              recs_per_artist: int=5):
  """Порекоммендовать пользователю исполнителей на основе его предпочтений

  Parameters
  :param user_id: int - ID пользователя, для которого делается рекоммендация
  :param fav_artist: int (default 5)  - Количество любимых исполнителей,
                                        на основе которых производится 
                                        рекоммендация
  :param recs_per_artist: int (default 5)  - Количество артистов
                                                        рекоммендуемых от 
                                                        каждого любимого
                                                        исполнителя

  Return
  :return result: set  - Список рекоммендуемых исполнителей
  """
  fav_artists = find_favorite_artists(user_id, fav_artists)
  result = []
  for artist in fav_artists:
    sim_artists = get_similar_artists_sparse(artist, recs_per_artist)
    result.extend(sim_artists)
  result = set(sklearn.utils.shuffle(result))
  return result

In [56]:
title_dict = {i : k for i, k in artists_data.values}
users_data.groupby('user_id').count().mean()

artist_id    49.044397
scrobbles    49.044397
dtype: float64

In [57]:
rows, r_pos = np.unique(users_data.values[:, 0], return_inverse=True)
cols, c_pos = np.unique(users_data.values[:, 1], return_inverse=True)

In [58]:
scrobbles_sparse = sparse.csr_matrix((
    conditional_users_artists_table['scrobbles'].to_numpy(), (r_pos, c_pos)))

Нормализация и получение матрицы схожести

In [59]:
Pui = normalize(scrobbles_sparse, norm='l2', axis=1)
sim = Pui.T * Pui

In [60]:
find_favorite_artists(1)

['Duran Duran', 'Morcheeba', 'Air', 'Hooverphonic', 'Kylie Minogue']

In [61]:
find_favorite_artists(2)

['Pleq', 'Segue', 'Max Richter', 'Celer', 'Pjusk']

In [62]:
get_similar_artists_sparse('The Beatles', 10)

['Led Zeppelin',
 'The Killers',
 'The Strokes',
 'Arctic Monkeys',
 'Muse',
 'Oasis',
 'Coldplay',
 'Pink Floyd',
 'Radiohead',
 'The Beatles']

In [63]:
get_similar_artists_sparse('Nirvana', 10)

['The Doors',
 "Guns N' Roses",
 'Red Hot Chili Peppers',
 'Metallica',
 'Foo Fighters',
 'Muse',
 'Green Day',
 'Radiohead',
 'The Beatles',
 'Nirvana']

In [64]:
get_recommendation_sparse(1)

{'Air',
 'Britney Spears',
 'Christina Aguilera',
 'Coldplay',
 'Depeche Mode',
 'Duran Duran',
 'Hooverphonic',
 'Kylie Minogue',
 'Lady Gaga',
 'Massive Attack',
 'Morcheeba',
 'New Order',
 'Pet Shop Boys',
 'Placebo',
 'Radiohead',
 'Regina Spektor',
 'Rihanna',
 'The Cure',
 'The Veronicas',
 'Ежи И Петруччо'}